# Initiale Datevorbereitung

## Ziel

Die Rohdaten sollen bereinigt und in Training- und Test-Set gespalten werden

Das beinhaltet:

* Daten in Reihenfolge bringen
* Fehlerhafte Features entfernen
* Design einer sinnvollen Zielvariable / Variationen von Zielvariablen
* Zusammenfügen der Datensätze
* Daten randomisieren
* Aufteilen in Training- und Test-Set


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone
import math

In [ ]:
plt.rcParams["figure.figsize"] = (50,40)

In [ ]:
failures_2016 = pd.read_csv("./data/init/failures-2016.csv", sep=";")
failures_2017 = pd.read_csv("./data/init/failures-2017.csv", sep=";")
metmast_2016 = pd.read_csv("./data/init/metmast-2016.csv", sep=";")
metmast_2017 = pd.read_csv("./data/init/metmast-2017.csv", sep=";")
signals_2016 = pd.read_csv("./data/init/signals-2016.csv", sep=";")
signals_2017 = pd.read_csv("./data/init/signals-2017.csv", sep=";")

## 1. Signale betrachten

### Signale beider Jahre kombinieren

In [ ]:
signals = pd.concat([signals_2016, signals_2017])

In [ ]:
signals.describe()

### Die Signale für jede Turbine aufbereiten und plotten

In [ ]:
turbine_names = signals["Turbine_ID"].unique()

In [ ]:
def create_df_for_each_turbine(signals):
    turbine_dfs = list();

    for turbine in turbine_names:
        test = signals["Turbine_ID"]
        turbine_df = signals[signals["Turbine_ID"] == turbine]
        turbine_df = turbine_df.sort_values("Timestamp")
        turbine_df = turbine_df.reset_index()
        turbine_dfs.append(turbine_df)

    return turbine_dfs

turbine_dfs = create_df_for_each_turbine(signals)


In [ ]:
def plot_data(df, title, n_agg, figsize=(10,9)):
    my_df = df.groupby(df.index // n_agg).agg("mean")
    fig, axs = plt.subplots(figsize[0], figsize[1])
    fig.suptitle(title)
    fig.tight_layout(pad=5.0)
    ctr2 = -1
    for i, col in enumerate(my_df):
        mod = i % figsize[0]
        if mod == 0: ctr2 +=1
        ax = axs[mod, ctr2]
        ax.scatter(x=my_df.index, y=my_df[col], s=1)
        ax.set_title(col)

In [ ]:
# for i, df in enumerate(turbine_dfs):
    # plot_data(df, turbine_names[i], 20)

### Signale mit niedriger Varianz herausfiltern

In [ ]:

def get_signals_with_low_variance(df, threshold=0):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(cont_data)
    inverted_list = ~np.array(selector.get_support())
    return cont_data.columns[inverted_list].tolist()

In [ ]:
cols_to_drop = set()
for i, df in enumerate(turbine_dfs):
    cols_to_drop.update(get_signals_with_low_variance(df))

### Signale mit hoher Korrealität finden

In [ ]:
def get_signals_with_high_correlation(df, threshold=0.5):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    corrM = cont_data.corr(min_periods=102921)
    arr = np.eye(len(corrM), dtype=bool)
    masked = corrM.mask(arr)
    abs = masked.abs()
    m = ~(abs < threshold).any()
    print(np.count_nonzero(m)) 
    return m

### Signale wieder zusammenführen

In [ ]:
signals_clean = pd.concat(turbine_dfs)
signals_clean = signals_clean.drop(list(cols_to_drop), axis=1)

In [ ]:
signals_clean.describe()

## 2. Mast-Daten betrachten

In [ ]:
metmast = pd.concat([metmast_2016, metmast_2017])

In [ ]:
metmast.head()

In [ ]:
df = metmast.describe()
df.loc['dtype'] = metmast.dtypes
df.loc['size'] = len(metmast)
df.loc['n_NaN'] = metmast.isnull().mean() * len(metmast)

print(df)

In [ ]:
metmast = metmast.sort_values("Timestamp")
metmast = metmast.reset_index()

plot_data(metmast, "Mast-Daten", 10, (6, 7))

In [ ]:
cols_to_drop = get_signals_with_low_variance(metmast)

In [ ]:
# clean automatic features
metmast_clean = metmast.drop(cols_to_drop, axis=1)
# clean broken sensor data
metmast_clean = metmast_clean.drop(["Min_Winddirection2", "Max_Winddirection2", "Avg_Winddirection2", "Var_Winddirection2"], axis=1)

In [ ]:
metmast_clean.describe()

In [ ]:
failures = pd.concat([failures_2016, failures_2017])

In [ ]:
failures.head()

In [ ]:
failures.describe()

# Datensätze in einen Datensatz zusammenführen
## Sensor- und Mastdaten mergen

In [ ]:
merged_df = pd.merge(signals_clean.reset_index(drop=True), metmast_clean.reset_index(drop=True), on="Timestamp", how="left")

In [ ]:
merged_df.describe(include="all")

In [ ]:
# for column in merged_df.columns:
#     if (pd.api.types.is_numeric_dtype(merged_df[column])):
#         merged_df[column].fillna(merged_df[column].median(), inplace = True)

In [ ]:
merged_df.drop(columns=["index_x"], inplace=True)

In [ ]:
merged_df.sample(5)

## Regression Targets vorbereiten und mergen

In [ ]:
failures_gearbox = failures[failures["Component"] == "GEARBOX"]
failures_gearbox.reset_index(drop=True, inplace=True); failures_gearbox

Da die labels nicht auf 10 Minuten gerundet sind (Wie Sensor- und Mastdaten), wird das an dieser Stelle getan damit die Labels mit den restlichen Daten kompatibel sind:

In [ ]:
def get_round_minute_diff(datetime_in: datetime) -> timedelta:
    min = datetime_in.minute
    rounded_min = round(min, -1)
    diff = rounded_min - min
    return timedelta(minutes=diff)

In [ ]:
def convert_round_minute_to_time(datetime_in: datetime) -> datetime:
    td = get_round_minute_diff(datetime_in)
    return datetime_in + td

Für die Regression-Labels, erzeugen die über <code>days_lookback</code> bis zum Fehlereintritt von 1 bis 0 abfallen. Für Klassifizierung werden die Labels im Zeitraum auf 1 gesetzt.
<br/>
<span style="color:red">TODO: Aktuell wird im Code nicht berücksichtigt, wenn sich labels überschneiden, da das für die Gearbox nicht der Fall ist.</span>

In [ ]:
days_lookback = 60

def create_failure_list(days_lookback: int, value_function, target_name: str = "Target") -> pd.DataFrame:
    
    ten_mins_of_n_days = int(24 * 60 * days_lookback / 10)
    failure_list = []
    for i, failure in enumerate(failures_gearbox):
        turbine_id = str(failures_gearbox["Turbine_ID"][i])
        failure_ts = str(failures_gearbox["Timestamp"][i])
        failure_datetime = datetime.fromisoformat(failure_ts)
        rounded_datetime = convert_round_minute_to_time(failure_datetime)
        for j in range(ten_mins_of_n_days):
            delta = timedelta(minutes=j*10)
            new_datetime = rounded_datetime - delta
            datetime_formated = new_datetime.replace(tzinfo=timezone.utc)
            failure_list.append([turbine_id, datetime_formated.isoformat(), value_function(j, ten_mins_of_n_days)])
    
    failure_df = pd.DataFrame(failure_list, columns=["Turbine_ID", "Timestamp", target_name])

    return failure_df


In [ ]:
reg_target_name = "RUL (Target)"
class_target_name = "Failure (Target)"

regression_function = lambda i, j: i / j
classif_function = lambda i, j: 1

failure_df_reg = create_failure_list(days_lookback, regression_function, reg_target_name)
failure_df_class = create_failure_list(days_lookback, classif_function, class_target_name)

In [ ]:
failure_df_class.head()

Der Feature-Datensatz wird mit den Labels zusammengeführt. Dabei ist besonders wichtig, dass der Bezug zu der jeweiligen Turbine bestehen bleibt.

In [ ]:
labeled_df_temp = pd.merge(merged_df.reset_index(drop=True), failure_df_reg.reset_index(drop=True), on=["Turbine_ID", "Timestamp"], how="left"); 
labeled_df = pd.merge(labeled_df_temp.reset_index(drop=True), failure_df_class.reset_index(drop=True), on=["Turbine_ID", "Timestamp"], how="left"); 

In [ ]:
labeled_df[reg_target_name].fillna(1.0, inplace = True)
labeled_df[class_target_name].fillna(0, inplace = True)

In [ ]:
labeled_df.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

In [ ]:
labeled_df[class_target_name].plot()

In [ ]:
labeled_df[reg_target_name].plot()

# Daten in Training und Test Set splitten

In [ ]:
# Alle Daten ab August 2017 liegen im Testset
split_criterion_reg = labeled_df["Timestamp"] >= "2017-08-00T00:00:00+00:00"

test_gearbox = labeled_df[split_criterion_reg].sample(frac=1).reset_index(drop=True)
train_gearbox = labeled_df[~split_criterion_reg].sample(frac=1).reset_index(drop=True)

In [ ]:

plt.scatter(x=test_gearbox.index, y=test_gearbox[reg_target_name])
# train_simple_regression_gearbox.describe()

# Erzeugte Daten abspeichern

In [ ]:
import os
os.makedirs("./data/first_clean", exist_ok=True)

In [ ]:
train_gearbox.to_csv("./data/first_clean/train_gearbox.csv")
test_gearbox.to_csv("./data/first_clean/test_gearbox.csv")